# EDA og Validering - Consulting Company Analytics

Dette notebook udfører exploratory data analysis og validerer datakvaliteten for consulting company dataset.

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set project root
PROJECT_ROOT = Path().resolve().parent
DB_PATH = PROJECT_ROOT / "analytics.db"
DATA_RAW = PROJECT_ROOT / "data" / "raw"

print(f"Project root: {PROJECT_ROOT}")
print(f"Database: {DB_PATH.exists()}")
print(f"Data directory: {DATA_RAW.exists()}")

## 1. Data Loading

In [ ]:
# Connect to database
conn = sqlite3.connect(DB_PATH)

# Load dimension tables
consultants = pd.read_sql_query("SELECT * FROM dim_consultant", conn)
clients = pd.read_sql_query("SELECT * FROM dim_client", conn)
projects = pd.read_sql_query("SELECT * FROM dim_project", conn)

# Load fact tables
timesheets = pd.read_sql_query("SELECT * FROM fact_timesheet", conn)
invoices = pd.read_sql_query("SELECT * FROM fact_invoice", conn)

print("Data loaded successfully!")
print(f"\nConsultants: {len(consultants)}")
print(f"Clients: {len(clients)}")
print(f"Projects: {len(projects)}")
print(f"Timesheet entries: {len(timesheets):,}")
print(f"Invoices: {len(invoices)}")

## 2. Data Quality Checks

In [ ]:
def check_data_quality(df, table_name):
    """Check for nulls, duplicates, and data types."""
    print(f"\n=== {table_name} ===")
    print(f"Shape: {df.shape}")
    print(f"\nNull values:")
    nulls = df.isnull().sum()
    print(nulls[nulls > 0] if nulls.sum() > 0 else "  No nulls")
    print(f"\nDuplicates: {df.duplicated().sum()}")
    print(f"\nData types:")
    print(df.dtypes)

# Check all tables
check_data_quality(consultants, "Consultants")
check_data_quality(clients, "Clients")
check_data_quality(projects, "Projects")
check_data_quality(timesheets, "Timesheets")
check_data_quality(invoices, "Invoices")

In [ ]:
# Check referential integrity
print("=== Referential Integrity Checks ===")

# Timesheets: consultant_id and project_id
invalid_consultants = timesheets[~timesheets['consultant_id'].isin(consultants['consultant_id'])]
invalid_projects = timesheets[~timesheets['project_id'].isin(projects['project_id']) & timesheets['project_id'].notna()]
print(f"Invalid consultant_ids in timesheets: {len(invalid_consultants)}")
print(f"Invalid project_ids in timesheets: {len(invalid_projects)}")

# Invoices: project_id
invalid_invoice_projects = invoices[~invoices['project_id'].isin(projects['project_id'])]
print(f"Invalid project_ids in invoices: {len(invalid_invoice_projects)}")

# Projects: client_id and project_manager_id
invalid_clients = projects[~projects['client_id'].isin(clients['client_id'])]
invalid_pms = projects[~projects['project_manager_id'].isin(consultants['consultant_id'])]
print(f"Invalid client_ids in projects: {len(invalid_clients)}")
print(f"Invalid project_manager_ids in projects: {len(invalid_pms)}")

## 3. Business Rules Validation

In [ ]:
# Convert dates
timesheets['work_date'] = pd.to_datetime(timesheets['work_date'])
projects['start_date'] = pd.to_datetime(projects['start_date'])
projects['end_date'] = pd.to_datetime(projects['end_date'])
invoices['invoice_date'] = pd.to_datetime(invoices['invoice_date'])

print("=== Business Rules Validation ===")

# Rule 1: Daily hours should be 6-8 on weekdays, mostly 0 on weekends
timesheets['is_weekend'] = timesheets['work_date'].dt.weekday >= 5
weekday_hours = timesheets[~timesheets['is_weekend']]['hours']
weekend_hours = timesheets[timesheets['is_weekend']]['hours']

print(f"\n1. Daily Hours Distribution:")
print(f"   Weekday hours - Min: {weekday_hours.min():.2f}, Max: {weekday_hours.max():.2f}, Mean: {weekday_hours.mean():.2f}")
print(f"   Weekend hours - Min: {weekend_hours.min():.2f}, Max: {weekend_hours.max():.2f}, Mean: {weekend_hours.mean():.2f}")
print(f"   Weekend entries: {len(weekend_hours)} ({len(weekend_hours)/len(timesheets)*100:.1f}%)")

In [ ]:
# Rule 2: Utilization by level
utilization = pd.read_sql_query("""
    SELECT 
        c.level,
        SUM(CASE WHEN t.billable_flag = 1 THEN t.hours ELSE 0 END) AS billable_hours,
        SUM(t.hours) AS total_hours,
        ROUND(SUM(CASE WHEN t.billable_flag = 1 THEN t.hours ELSE 0 END) * 100.0 / SUM(t.hours), 2) AS utilization_pct
    FROM fact_timesheet t
    JOIN dim_consultant c ON t.consultant_id = c.consultant_id
    GROUP BY c.level
""", conn)

print("\n2. Utilization by Level:")
print(utilization.to_string(index=False))

In [ ]:
# Rule 3: Project dates should be valid
invalid_dates = projects[projects['end_date'] < projects['start_date']]
print(f"\n3. Project Date Validation:")
print(f"   Projects with end_date < start_date: {len(invalid_dates)}")

# Rule 4: Project durations
projects['duration_days'] = (projects['end_date'] - projects['start_date']).dt.days
print(f"\n4. Project Duration:")
print(f"   Min: {projects['duration_days'].min()} days")
print(f"   Max: {projects['duration_days'].max()} days")
print(f"   Mean: {projects['duration_days'].mean():.1f} days")

In [ ]:
# Rule 5: Payment days by sector
payment_by_sector = pd.read_sql_query("""
    SELECT 
        cl.sector,
        AVG(i.payment_days) AS avg_payment_days,
        COUNT(*) AS invoice_count
    FROM fact_invoice i
    JOIN dim_project p ON i.project_id = p.project_id
    JOIN dim_client cl ON p.client_id = cl.client_id
    GROUP BY cl.sector
""", conn)

print("\n5. Payment Days by Sector:")
print(payment_by_sector.to_string(index=False))

## 4. Exploratory Data Analysis

In [ ]:
# Consultant distribution
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Level distribution
consultants['level'].value_counts().plot(kind='bar', ax=axes[0, 0], color='steelblue')
axes[0, 0].set_title('Consultants by Level')
axes[0, 0].set_ylabel('Count')
axes[0, 0].tick_params(axis='x', rotation=45)

# Role distribution
consultants['role'].value_counts().plot(kind='bar', ax=axes[0, 1], color='coral')
axes[0, 1].set_title('Consultants by Role')
axes[0, 1].set_ylabel('Count')
axes[0, 1].tick_params(axis='x', rotation=45)

# Country distribution
consultants['country'].value_counts().plot(kind='bar', ax=axes[1, 0], color='lightgreen')
axes[1, 0].set_title('Consultants by Country')
axes[1, 0].set_ylabel('Count')
axes[1, 0].tick_params(axis='x', rotation=45)

# Cost rate distribution
consultants['cost_rate_dkk_per_hour'].hist(bins=20, ax=axes[1, 1], color='gold')
axes[1, 1].set_title('Cost Rate Distribution')
axes[1, 1].set_xlabel('DKK per Hour')
axes[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Client and project analysis
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Client sector distribution
clients['sector'].value_counts().plot(kind='pie', ax=axes[0, 0], autopct='%1.1f%%')
axes[0, 0].set_title('Clients by Sector')
axes[0, 0].set_ylabel('')

# Project contract type
projects['contract_type'].value_counts().plot(kind='bar', ax=axes[0, 1], color='teal')
axes[0, 1].set_title('Projects by Contract Type')
axes[0, 1].set_ylabel('Count')
axes[0, 1].tick_params(axis='x', rotation=45)

# Project budget distribution
projects['budget_value_dkk'].hist(bins=30, ax=axes[1, 0], color='purple')
axes[1, 0].set_title('Project Budget Distribution')
axes[1, 0].set_xlabel('Budget (DKK)')
axes[1, 0].set_ylabel('Frequency')

# Project duration distribution
projects['duration_days'].hist(bins=30, ax=axes[1, 1], color='orange')
axes[1, 1].set_title('Project Duration Distribution')
axes[1, 1].set_xlabel('Duration (Days)')
axes[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Timesheet and utilization trends
monthly_util = pd.read_sql_query("""
    SELECT 
        strftime('%Y-%m', work_date) AS month,
        SUM(billable_hours) AS billable_hours,
        SUM(total_hours) AS total_hours,
        AVG(utilization_pct) AS avg_utilization_pct
    FROM v_utilization_daily
    GROUP BY strftime('%Y-%m', work_date)
    ORDER BY month
""", conn)

monthly_util['month'] = pd.to_datetime(monthly_util['month'] + '-01')

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Monthly hours
axes[0].plot(monthly_util['month'], monthly_util['billable_hours'], marker='o', label='Billable Hours', linewidth=2)
axes[0].plot(monthly_util['month'], monthly_util['total_hours'], marker='s', label='Total Hours', linewidth=2)
axes[0].set_title('Monthly Hours Trend')
axes[0].set_ylabel('Hours')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].tick_params(axis='x', rotation=45)

# Monthly utilization
axes[1].plot(monthly_util['month'], monthly_util['avg_utilization_pct'], marker='o', color='green', linewidth=2)
axes[1].axhline(y=75, color='r', linestyle='--', label='Target (75%)')
axes[1].set_title('Monthly Average Utilization %')
axes[1].set_ylabel('Utilization %')
axes[1].set_xlabel('Month')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Revenue and financial analysis
monthly_revenue = pd.read_sql_query("""
    SELECT 
        strftime('%Y-%m', invoice_date) AS month,
        SUM(amount_dkk) AS total_revenue,
        SUM(CASE WHEN paid_flag = 1 THEN amount_dkk ELSE 0 END) AS paid_revenue
    FROM fact_invoice
    GROUP BY strftime('%Y-%m', invoice_date)
    ORDER BY month
""", conn)

monthly_revenue['month'] = pd.to_datetime(monthly_revenue['month'] + '-01')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Monthly revenue
axes[0].bar(monthly_revenue['month'], monthly_revenue['total_revenue'] / 1e6, 
            label='Total Revenue', alpha=0.7, color='steelblue')
axes[0].bar(monthly_revenue['month'], monthly_revenue['paid_revenue'] / 1e6, 
            label='Paid Revenue', alpha=0.7, color='green')
axes[0].set_title('Monthly Revenue Trend')
axes[0].set_ylabel('Revenue (Million DKK)')
axes[0].set_xlabel('Month')
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')
axes[0].tick_params(axis='x', rotation=45)

# Payment status
payment_status = invoices['paid_flag'].value_counts()
axes[1].pie(payment_status.values, labels=['Unpaid', 'Paid'], autopct='%1.1f%%', 
            colors=['coral', 'lightgreen'])
axes[1].set_title('Invoice Payment Status')

plt.tight_layout()
plt.show()

## 5. Key Insights Summary

In [ ]:
# Calculate key metrics
total_revenue = invoices['amount_dkk'].sum()
paid_revenue = invoices[invoices['paid_flag'] == 1]['amount_dkk'].sum()
total_billable_hours = timesheets[timesheets['billable_flag'] == 1]['hours'].sum()
total_hours = timesheets['hours'].sum()
overall_utilization = (total_billable_hours / total_hours * 100) if total_hours > 0 else 0

# Projects over budget
project_financials = pd.read_sql_query("SELECT * FROM v_project_financials", conn)
over_budget_projects = project_financials[project_financials['actual_hours'] > project_financials['budget_hours']]

print("=== KEY INSIGHTS ===")
print(f"\n1. Financial Overview:")
print(f"   Total Revenue: {total_revenue:,.0f} DKK")
print(f"   Paid Revenue: {paid_revenue:,.0f} DKK ({paid_revenue/total_revenue*100:.1f}%)")
print(f"   Outstanding AR: {total_revenue - paid_revenue:,.0f} DKK")

print(f"\n2. Utilization:")
print(f"   Overall Utilization: {overall_utilization:.1f}%")
print(f"   Total Billable Hours: {total_billable_hours:,.0f}")
print(f"   Total Hours: {total_hours:,.0f}")

print(f"\n3. Project Health:")
print(f"   Total Projects: {len(projects)}")
print(f"   Projects Over Budget: {len(over_budget_projects)} ({len(over_budget_projects)/len(projects)*100:.1f}%)")
if len(over_budget_projects) > 0:
    print(f"   Average Overrun: {over_budget_projects['hours_overrun_pct'].mean():.1f}%")

print(f"\n4. Client Base:")
print(f"   Total Clients: {len(clients)}")
print(f"   Public Sector: {len(clients[clients['sector'] == 'Public'])}")
print(f"   Private Sector: {len(clients[clients['sector'] == 'Private'])}")

conn.close()